<a href="https://colab.research.google.com/github/spttleonardo/CNN-application/blob/main/Questao2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Alunos:** Leonardo Schmitt e Bruno Yuri

**Atividade 2**- Necessário criar um classificador de imagens utilizando redes neurais. Deve-se  definir e treinar uma rede neural convolucional que seja capaz de indetificar um conjunto de imagens e destinguir se a mesma trata-se  de um gato ou uma cobra.

- Dataset utilizado: Autoria (Disponivel no Drive)

Deve-se obter uma acurácia > 90% em um dataset de teste (não utilizado para treinamento e validação)
Além disso o trabalho deve Apresentar a matriz de confusão paraanálise dos acertos e erros em cada classe
individualmente.


**1-** Para fazer o download do arquivo disponibilizado no drive, deve-se usar um programar chamado gdown que precisa ser instalado no ambiente. No colab, as instruções precedidas por ! são executadas como comandos do sistema operacional (no caso do colab é Linux).

In [8]:
!pip install gdown

**2**- Uma vez instalado o gdown, deve-se usar ele para fazer downloado do arquivo dataset_pequeno.zip de um compartilhamento público (compartilhamento acessível apenas com o link) do GoogleDrive.

Para obter o arquivo com o gdown, é necessário colocar o arquivo no GoogleDrive, compartilhar ele com link e extrair do link apenas o id do compartilhamento. Por exemplo, para o compartilhamento da URL abaixo:

https://drive.google.com/drive/folders/1uYBF5a4IqnmmVIJr-UsGYmUs0tKymOAY?usp=sharing

o id do compartilhamento é: 1uYBF5a4IqnmmVIJr-UsGYmUs0tKymOAY

então basta substituir na URL abaixo que será usada pelo gdown o texto id_do_compartilhamento pelo id obtido:

gdown https://drive.google.com/uc?id=id_do_compartilhamento

Abaixo o uso do gdown para este arquivo (dataset_pequeno.zip) com alguns comandos adicionais comentados no texto. Caso julgue interessante, separe cada comando em uma caixa para executá-lo em separado e ver o resultado que ele produz:

In [ ]:
#Remover o dataset_pequeno.zip caso já exista
!rm -rfv dataset_pequeno/

#Remover o dirtório gerado ao descompactar o exemplo.zip caso já existe
!rm -rfv dataset_pequeno/

#Baixar o arquivo do GoogleDrive usando o gdown
#https://drive.google.com/file/d/139g4a_E9rsZmBUGE82Sj0bIhOQxkEuAM/view?usp=sharing-  dataset com menos exemplos
!gdown  https://drive.google.com/uc?id=139g4a_E9rsZmBUGE82Sj0bIhOQxkEuAM

#Descompactar o arquivo exemplo.zip
!unzip dataset_pequeno.zip

#Lista arquivos e diretorios
!ls -ltr

3 - Lendo os arquivos de entrada usando o ImageDataGenerator.

A classe ImageDataGenerator que é parte da API do Keras é um componente de grande utilidade para carregar e fazer pré-processamento das imagens para uma rede neural programada com o Keras.

Além da função de ler os arquivos contendo as imagens com base em uma organização de diretórios, ele possuí outras funcionalidades de pré-processamento, como redimensionamento, rotação, zoom, resampling, etc. Para mais detalhes visite: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator.

Além do componente ImageDataGenerator, existem outros componentes e métodos para ler os dados de imagens e transformá-las em matrizes no formato necessário para usá-las em uma rede CNN. Um exemplo de outro componente é o [tf.keras.preprocessing.image_dataset_from_directory](https://keras.io/api/preprocessing/image/) entre outros.

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definindo o tamanho (tam x tam) que sera usado para as imagens na CNN
tam = 100

gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                         rotation_range=7,
                                         horizontal_flip=True,
                                         shear_range=0.2,
                                         height_shift_range=0.07,
                                         zoom_range=0.2)

# Usando o objeto gerador_treinamento criado, para carregar os dados de treino do dataset
base_treinamento = gerador_treinamento.flow_from_directory('dataset_pequeno/dataset/training_set',
                                                           target_size=(tam,tam),
                                                           batch_size=1000,
                                                           class_mode='binary')

# Normalizacao dos valores
gerador_teste = ImageDataGenerator(rescale=1./255)

# Realizando o carregamento das imagens de validação
base_validacao = gerador_teste.flow_from_directory('dataset_pequeno/dataset/valid_set',
                                                    target_size=(tam,tam),
                                                    batch_size=1000,
                                                    class_mode='binary')

# Fazendo a carregamento  das imagens de teste
base_teste = gerador_teste.flow_from_directory('dataset_pequeno/dataset/test_set',
                                                target_size=(tam,tam),
                                                batch_size=1000,
                                                class_mode='binary')


Found 600 images belonging to 2 classes.
Found 240 images belonging to 2 classes.
Found 360 images belonging to 2 classes.


**4-** Importando módulos e definindo a rede neural convolucional.

Para definir a rede neural convolucional e realizar a comparação das métricas graficamente, primeiramente deve-se importar os módulos necessários.

In [11]:
import numpy as np

from tensorflow import keras
from matplotlib import pyplot
from keras import optimizers


import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


Definindo a rede neural utilizando a função *Sequencial()* do keras.

Vale notar que utiliza-se o dropout de 25% para que nao ocorra overfitting.


In [12]:
model = keras.Sequential(
    [
        keras.Input(shape=(tam,tam,3)),
        keras.layers.Conv2D(32, kernel_size=(3,3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(8, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 98, 98, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 47, 47, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 33856)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 33856)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │         270,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 290,257 (1.11 MB)

 Trainable params: 290,257 (1.11 MB)

 Non-trainable params: 0 (0.00 B)

**5-** Definindo parâmetros para o treinamento da CNN.

O valor de epoca deve ser deixar em 150, para que o mesmo consiga atingir um valor aceital de acuracia tanto para treinamento como para teste. Além disso, a taxa de aprendizado foi ajustada para 0.03 para que melhore o resultados obtidos pelada rede neural.

Destaca-se ainda que o batch_size tem como valor 8, para melhor desempenho da rede neural.

In [ ]:
epochs = 150
optimizer_exp = keras.optimizers.Adam(learning_rate=0.03)
#SGD = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)
model.compile(loss="binary_crossentropy", optimizer=optimizer_exp, metrics=["accuracy"])

hist = model.fit(base_treinamento,
                           validation_data=base_validacao,
                           epochs=epochs,batch_size=8)


**6-** Plotagem do gráficos para realizar a comparação entre a acurácia e loss do dados de treinamento e validação

In [ ]:
print(hist.history.keys())

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model_accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','valid'], loc='upper left')
plt.show()


plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'legend'], loc='upper left')
plt.show

**7-** Seção de codigo para valiar o desempenho da CNN com os dados de teste

In [ ]:
score = model.evaluate(base_teste, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

**8-** Realização de previsões, plotagem da matriz de confusão e avaliação da acurácia.

Nesta seção, busca-se demonstrar como a CNN se comporta ao realizar previsões e como validar os dados previstos por meio da matriz de confusão e da acurácia. Essas etapas são cruciais para avaliar o desempenho do modelo em dados de teste não vistos.

In [ ]:
images,labels=next(base_teste)
print(images.shape)
print(labels.shape)

preditcions = (model.predict(base_teste, verbose=0)>0.5).astype(int)

print('Matriz de confusão')
print(confusion_matrix(labels, preditcions))

print('Accuracy: %f' %(accuracy_score(labels, preditcions)))

plt.figure(figsize=(20, 20))
for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(images[i])
    plt.title(list(base_teste.class_indices.keys())[labels[i].astype(int)] + "->" + list(base_teste.class_indices.keys())[preditcions[i,0].astype(int)])
    plt.axis("off")

print(np.column_stack((labels, preditcions)))


Destarte, conclui-se que o objetivo pode ser alcançado pera as configurações da rede neural presente. Além disso, a forma como foi definido os parâmetros são o suficiente para poder alcançar um acurácia acima de 90%.

Entretanto, devido ao ruido presente nas imagens, como também do mal condicionamento das imagens, pode acontecer de a acurácia ficar abaixo dos 90%.